#### Top tips for primitive types:

- Be very comfortable with **bitwise operators**, particulary XOR
- Understand how to use **masks** annd create them in an **machine independent way**
- Know fast ways to **clear the lowermost set bit**(and by extension, set the lowermost 0, get its index, etc.)
- Understand **signedness** and its implication to **shifting**
- Consider using a **cache** to accelerate operations by using it to brute-force small inputs.

In [1]:
# Bitwise operators

a = 10  # 1010
b = 7   # 0111

# AND
print(a & b)

# OR
print(a | b)

# XOR 1010 ^ 0111 = 1101  = 13
print(a ^ b)

# NOT (1's complement) -(1010 +1) = -11
print(~a)

# RIGHT SHIFT 1010 = 0101
print(a>>1)

# LEFT SHIFT 1010 = 10100
print(a<<1)


2
15
13
-11
5
20


### Program 1
Write a program to count the number of bits that are set to 1 in a nonnegative integer.

In [2]:
def count(x):
    num_bits=0
    while x:
        num_bits += x & 1  
        x>>=1   #shorthand 
    return num_bits

print(count(100))

3


Integers is Python are **unbounded** - the maximum integer representable is a function of the available memory.

In [3]:
import sys

# maximum value integer that can be stored in a word. 2**63 -1 on 64-bit machine
print(sys.maxsize)

# Bounds on float
print(sys.float_info) 

9223372036854775807
sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


**Negative numbers** are treated as their 2's complement value. (There is no concept of an unsigned shift in python, since integers have infinite precision

2's complement is 1 added to 1's complement

In [4]:
number = 15

# convert integer into binary format. ref:https://stackoverflow.com/a/10411108/21022287
#binary_number = int("{0:08b}".format(number))
# Alternative way
binary_number = int(format(number,"08b"))
print("binary_number: ", binary_number)

# 1's complement
flipped_binary_number = ~ binary_number
print("flipped_binary_number: ", flipped_binary_number)

# 2's complement
flipped_binary_number = flipped_binary_number + 1
print("added one: ", flipped_binary_number)

# convert integer into string
str_twos_complement = str(flipped_binary_number)
print("string: ", str_twos_complement)

#convert string into decimal value. ref:https://www.programiz.com/python-programming/methods/built-in/int
twos_complement = int(str_twos_complement, 2)
print("integer: ", twos_complement)


binary_number:  1111
flipped_binary_number:  -1112
added one:  -1111
string:  -1111
integer:  -15


fast way to **Clear the lowermost set bit** (by extension, set the lowermost 0, get its index, etc) 

> Fastest way to find 2s-complement of a number is to get the rightmost set bit and flip everything to the left of it.
- eg: consider a 4 bit system
- 4=0100
- 2s complement of 4 = 1100, which nothing but -4
- 4&(-4)=0100.
- Notice that there is only one set bit and its at rightmost set bit of 4
- Similarly we can generalise this for n.
- n&(-n) will contain only one set bit which is actually at the rightmost set bit position of n.
- since there is only one set bit in n&(-n) , it is a power of 2.
- So finally we can get the bit position by:
- log2(n&(-n))+1

In [5]:
def clear_lowermost_set_bit(x):
    return x & x-1
print(clear_lowermost_set_bit(7)) # 0111 & 0110 = 0110
print(clear_lowermost_set_bit(6)) # 0110 & 0101 = 0100
print(clear_lowermost_set_bit(5)) # 0101 & 0100 = 0100
print(clear_lowermost_set_bit(4)) # 0100 & 0011 = 0000


# https://stackoverflow.com/a/42747608/2102228
import math
def get_first_set_bit(x):
    #return math.log2(x&-x)+1
    return x&-x 
    #return x & ~(x-1)
    #return x & (~x + 1)

print(get_first_set_bit(7)) # 8bit system: 7&-7 = 00000111 &  00000001 = 00000001 = 1 
print(get_first_set_bit(6)) # 8bit system: 6&-6 = 00000110 &  00000010 = 00000010 = 2

6
4
4
0
1
2


In [6]:
# key methods of numeric types
print(abs(-34.5))
print(math.ceil(2.17))
print(math.floor(2.17))
x = 8
y = -8
print(min(x,-4))
print(max(3.14,y))
print(pow(2.17,3.14)) # alternatively 2.17 ** 3.14
print(math.sqrt(225))



34.5
3
2
-4
3.14
11.38894680008054
15.0


In [7]:
# intercovert integer and strings
print(str(42))
print(int("42"))
print(str(3.14))
print(float("3.14"))



42
42
3.14
3.14


In [8]:
# Unlike integers, float are not infinite precision, and it is convenient to refer to infinity as
print(float('inf'))
print(float('-inf'))
# above values are comparable to integers, and can be used to create pseudo max-int and pseudo min-int

inf
-inf


In [9]:
# comparing float values, it is robust and can handle absolute and relative differences
print(math.isclose(1.233, 1.4566))
print(math.isclose(1.233, 1.233))
print(math.isclose(1.233, 1.233000001))

False
True
True


In [10]:
# key methods in random : https://www.w3schools.com/python/module_random.asp
import random
print(random.randrange(28))
print(random.randint(8,16))
print(random.random())
A = [32,23,43,46,66,98]
random.shuffle(A)
print(A)
print(random.choice(A))

4
12
0.3998973564611594
[23, 32, 46, 98, 43, 66]
66


## 4.1 COMPUTING THE PARITY OF A WORD

The parity of a binary word is 1 if the number of 1s in the word is odd; otherwise, it is 0 e.g. 1011 - parity is 1 and for 1010 - parity is 0

How would you compute the parity of very large number of 64-bit words? i.e 2^64 

### Program 2

In [11]:
# Brute-force

def parity(x):
    result = 0
    while x:
        result ^= x&1
        x>>=1
    return result

print(parity(10))
print(parity(11))

# time-complexity is O(n)

0
1


### Program 3
Using the logic to remove lowest-most bit to improve time complexity. logic: (x & x-1)

In [12]:
# First Improvement

def parity(x):
    result = 0
    while x:
        result ^=1
        x &= x-1
    return result

print(parity(10))
print(parity(11))

# time-complexity is O(k), where k is the number of one's


0
1


### Program 4
Computing the parity of very **large number of words**, When you have to perform a large number of parity computations, and, more generally, any kind of bit fiddling computations, two key to performance are **processing multiple bits at a time and caching results in an array-based lookup table**

- Clearly, we cannot keep the parity of every 64-bit integers - we would need 2^64 bits of storage (exabytes)
- However, we can compute the parity of a 64-bit integer by grouping its bits into four non-overlapping 16-bit subwords
- computing parity of each subwords, and then computing the parity of these four subresults.
- 2^16 = 65536 is relatively small, which makes it feasible ti cache the parity of all 16-bits words using an array,
- Furthermore, since 16 evenly divides 64, the code is simpler than we were, for e.g. to use 10 bit subwords

Let's illustrate this example lookup table of 2-bit words. 
- The cache is <0,1,1,0> these are parities of (00),(01),(10),(11) respectively.
- To compute the parity of (11001010), we would compute the parities of (11),(00),(10),(10)
- by table lookup, we see these are 0,0,1,1 respectively,
- so the final result is the parity of 0,0,1,0 which is 0

Algo
- To lookup the parity of first two bit in (11101010), we right shift by 6, to get (00000011), and use this index into the cache.
- To lookup the parity of the next two bits, i.e. (10) we right shift by 4, to get (00001110), to get(10) in the two least significant bit places. The right shift doesnot remove the leading 11 -(00001110), so we cannot index the cache with this, it leads to an OUT_OF_BOUND access.
- To get the last two bits after the right shift by 4, we bitwise AND (00001110) with (00000011) - this is the mask used to extract the last 2-bits. the result is (00000010). Similar masking is needed for other two 2-bit lookups.

**TODO: Implement a working code**

In [13]:
def parity(x):
    MASK_SIZE = 16
    BIT_MASK = 0xFFFF
    return(PRECOMPUTED_PARITY[x >> (3* MASK_SIZE)] ^
          PRECOMPUTED_PARITY[(x >>(2 * MASK_SIZE)) & BIT_MASK] ^
          PRECOMPUTED_PARITY[(x >> MASK_SIZE) & BIT_MASK] ^
          PRECOMPUTED_PARITY[x & BITMASK])

In [14]:
sixtyfour_bit = 18446744073709551616
#print(parity(sixtyfour_bit)) 

# TODO : implement this fully ..

the time complexity is a function of the size of the keys used to index the lookup tabke, Let L be the width of the words for which we cache the results, and n the word size. Since there are n/L terms, the **time complexity is O(n/L)** assuming word-level operations, such as shifting, take O(1) time. (This does not include the time for inititalization of the lookup table.)

We can improve on the O(n) worst-cast time complexity of the previous algorithms by exploiting some simple properties of parity. Specifically, the XOR of two bits is defined to be 0 if both bits are 0 or both bits are 1; otherwise it is 1.

XOR - associative i.e. it doesn't matter how we group bits
XOR - commutative i.e. the order in which we perform XORs doesnot change the result.

The **XOR of a group bit is it parity**, we can explot this fact to use the CPU's word-level XOR instruction to process multiple bits at a time.


In [15]:
def parity(x):
    x ^= x >> 32
    x ^= x >> 16
    x ^= x >> 8
    x ^= x >> 4
    x ^= x >> 2
    x ^= x >> 1
    return x & 0x1

print(parity(6))
print(parity(18446744073709551614))


# time complexity is O(log n), where n is the word-size

0
1


Note: We can combine caching with word-level operations, e.g. by doing a lookup in the XOR-based approach once we get to 16 bits.

The actual runtimes depend on the input data, e.g. the refinement of the brute-force algorithm is very fast on sparse inputs. However, for random inputs, the refinement of the brute-force is roughly **20% faster** than the brute-force algorithm. The Table based approach is **four times faster** still, and using associativity reduces run time by another factor of two.

### Program 5
**Variant:** Write expression that use bitwise operators, equality checks, and Boolean Operators to do the following in O(1) time

- Right propogate the rightmost set bit in x, e.g., turns (01010000)2 to (01011111)2 
- Compute x mod a power of two, e.g., return 13 for 77 mod 64
- Test if x is a power of 2, i.e. evaluates to true for x = 1,2,4,8,.. false for all other values.

In [16]:
# input 01010000 = 80
# output 01011111 = 95
# rigmost set bit in input corresponds to 16(10000), so 15(01111) will be all one's 
# 01010000
# 01001111
# 01011111 (OR)
def rightpropogate(x):
    return x | (x-1)

print(rightpropogate(80))

95


In [17]:
# Compute n modulo d without division(/) and modulo(%) operators, where d is a power of 2 number.
# 77 (01001101) mod 64 (01000000)= 13 (00001101)
# 01001101 77 
# 00111111 63 
# 00001101 13
def mod(x,y):
    return x & (y-1)

print(mod(77,64))

13


In [18]:
# 00000100 (4)
# 00000011 (3)
def PowerOfTwo(x):
    return (x & (x -1) == 0) and x != 0

print(PowerOfTwo(63))
print(PowerOfTwo(4))
print(PowerOfTwo(0))

False
True
False


## 4.2 SWAP BITS

A brute-force approach would be to use bitmasks to extract the ith and jth bits, saving them in local variables. Consequently, write the saved jth bit to index i and saved ith bit to index jm using combination of bitmask and bitwise operations

The brute-force approach works generally, e.g., if we were swapping objects stored in an array. However, since a bit can only take two values, we can do a little better. First check if bits two be swapped differ, if YES, swapping them is the same as flipping their individual values.

### Program 6

In [19]:
# swap bit at index i with bit at index j

# 64: 01000000
# i = 6, j =1
#     01000000
#     00000010
# OR  01000010 = 66
# XOR 00000010 = XOR of 64 and 66

def swapbits(x,i,j):
    if (x>>i & 1) != (x>>j & 1): 
        # first checking that both are not same, othewise there is no point in swaping
        # select the bits to flip with bit_mask, since x^1 = 0 when x =1 and 1 when x = 0
        # when x =0 , we can perform flip XOR
        bit_mask = (1 <<i) | (1 << j)
        x^= bit_mask
    return x

print(swapbits(64,6,1))

#timecomplexity O(1), independent of the word size

2


## 4.3 REVERSE BITS

Write a program that takes a 64-bit unsigned integer and returns the 64-bit unsigned integer consisting of the bits of the input in reverse order. For example, if the input is (1110000000000001), the output should be(1000000000000111).

Hint: Use a lookup table.

**Solution 1**: If we need to perform this operation just once, there is a simple brute-force algorithm: iterate through the 32 least significant bits of the input, and swap each with the corresponding most significant bit, using, swap bits (program 6)

**Solution 2**: To implement reverse when the operation is to be performed repeatedly, we look more carefully at the structure of the input, with an eye towards using a cache. Similar to computing parity using lookup, a very fast way to reverse bits of 16-bit integer when we are performing many reverses is to build an array-based lookup-table A such that every 16-bit integer y, A[y] holds the bit-reversal of y. 
e.g for 8-bit integers and 2-bit lookup table keys. The table is rev = <(00),(10),(01),(11)>. If the input is (10010011), its reverse is rev(11), rev(00), rev(01), rev(10), i.e., (11001001)


### Program 7
**TODO: Implement lookup table to complete following program. use 8-bits**


In [27]:
def reverse_bits(x):
    MASK_SIZE = 16
    BIT_MASK = 0xFFFF
    return (PRECOMPUTED_REVERSE[x & BIT_MASK] << (3 * MASK_SIZE) 
            | PRECOMPUTED_REVERSE[(x >> MASK_SIZE) & BIT_MASK ] << (2 * MASK_SIZE) 
            | PRECOMPUTED_REVERSE[(x >> (2 * MASK_SIZE)) & BIT_MASK ] <<  MASK_SIZE 
            | PRECOMPUTED_REVERSE[(x >> (3 * MASK_SIZE)) & BIT_MASK ])
            
# time-complexity  O(n/L), for n-bit integers and L-bit cache keys.

## 4.4 FIND A CLOSET INTEGER WITH THE SAME WEIGHT

Define the _weight_ of a non-negative integer x to be the number of bits that are set to 1 in its binary representation. for example, since 92 in base-2 equals (1011100), the weight of 92 is 4

Write a program which takes as input a nonnegative integer x and returns a number y which is not equal to x, but has the same weight as x and their difference, |y-x|, is as small as possible. You can assume x is not 0, or all 1s. For example, if x ==6, you should return 5. You can assume the integers fits in 64-bits.

Hint: Start with the least significant bit.

**Solution 1:**  A brute-force approach to try all the integers x-1, x+1, x-2, x+2... stopping as soon as we encounter one with the same weight at. It works poorly on some value for e.g. for 8, it will find in order 7,9,6,10,5,11,4 stopping at 4. The algorithm tries 2^(3-1) numbers smaller than 8, i.e. 7,6,5,4 and 2^(3-1) -1 i.e. 9,10,11. The example generalizes , suppose x = 2^30, the algorithm will evaluate between 2^30 and 2^29 + 2^30 and 2^29 -1 i.e. over one billion integers.

**Wrong Approach** A natural focus is to swap LSB with rightmost bit that differs from it. It works for some input but can result in wrong output. for e.g. 111 (7 in decimal), it return (1110) i.e. 14. however 1011 (11) has same weight and closer to 111.

**Solution 2:** Suppose we flip the bit at index **k1** and flip the bit at index **k2**, k1>k2. The absolute difference between orginal value and new value is 2^k1 - 2^k2. **To minimize this,** we should make k1 as small as possible and k2 as close to k1. 
Also to preserve weight, the bit at k1 has to be different from the bit at k2, otherwise the flips leads to different  weight. This means the smallest k1 is the rightmost bit that's different from the LSB, and k2 must be the very next bit. In Summary, **Swap the two rightmost consecutive bits that differ**